In [43]:
# Import libraries and modules
import csv  # for reading and writing CSV files
import matplotlib.pyplot as plt  # for creating plots and charts
import numpy as np  # for numerical computing with Python
import pandas as pd  # for data manipulation and analysis
import pathlib  # for interacting with file paths in a cross-platform manner
import tensorflow as tf  # for machine learning and deep learning
from keras.layers import Dense, Activation  # for building deep learning models in TensorFlow
from keras.models import Sequential  # for building deep learning models in TensorFlow
from keras.optimizers import Adam, RMSprop  # for building deep learning models in TensorFlow
from keras.callbacks import History
from sklearn.model_selection import train_test_split  # for model selection and evaluation
from sklearn.preprocessing import MinMaxScaler  # for preprocessing data
from tensorflow import keras  # for building deep learning models in TensorFlow
from tensorflow.keras import layers  # for building deep learning models in TensorFlow


# Print TensorFlow version
print(tf.__version__)

# Define column names for the CSV file
column_names = ['Open Time','Open','High','Low','Close',
                'Close Volume', 'Time', 'Quote asset volume',
               'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
               

# Read in the CSV file using pandas
raw_dataset = pd.read_csv("main.csv", names=column_names,
                      na_values = "?", comment='\t', dtype='float',
                      sep=",", skipinitialspace=True)

# Copy the data from the raw dataset to a new dataframe
dataset = raw_dataset.copy()

# Print the last few rows of the dataset
dataset.tail()


2.11.0


,Open Time,Open,High,Low,Close,Close Volume,Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
188312,1.620794e+12,57517.42,57526.28,57485.00,57485.07,42.575735,1.620794e+12,2.448258e+06,1195.0,15.319691,880913.090845
188313,1.620794e+12,57485.07,57496.42,57466.75,57481.49,34.205467,1.620794e+12,1.966194e+06,1096.0,15.971891,918058.816162
188314,1.620794e+12,57477.18,57509.99,57458.18,57470.00,30.211789,1.620794e+12,1.736514e+06,955.0,13.054229,750364.577319
188315,1.620794e+12,57470.00,57470.01,57400.00,57450.90,45.354728,1.620794e+12,2.605080e+06,1559.0,12.615628,724559.233035
188316,1.620794e+12,57450.89,57475.66,57435.51,57450.19,14.168318,1.620794e+12,8.140594e+05,730.0,7.247751,416412.022206


In [44]:
dataset.pop("Open Time")
dataset.pop("Time")
# Randomly select 80% of the rows from the dataset and store them in a new dataframe
train_dataset = dataset.sample(frac=0.8,random_state=0)

# Remove the rows in the training dataset from the original dataset, leaving the remaining rows in a new dataframe
test_dataset = dataset.drop(train_dataset.index)


In [45]:
# Calculate statistical summary of the training dataset
train_stats = train_dataset.describe()

# Remove the "Close" column from the statistical summary
train_stats.pop("Close")

# Transpose the statistical summary so that it's in a more useful shape
train_stats = train_stats.transpose()

In [46]:
# Remove the "Close" column from the training dataset and store it in a new dataframe
train_labels = train_dataset.pop('Close')

# Remove the "Close" column from the testing dataset and store it in a new dataframe
test_labels = test_dataset.pop('Close')

In [47]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_dataset)
normed_train_dataset = scaler.transform(train_dataset)
normed_test_dataset = scaler.transform(test_dataset)

In [48]:
print(normed_train_dataset.shape)
print(train_labels.shape)

(150654, 8)
(150654,)


In [49]:
print(train_labels)

54176     38273.80
22054     36653.35
25894     36570.67
3604      34482.99
96019     50880.06
            ...   
58124     46408.25
114876    57751.49
89163     51009.62
155465    56929.00
90475     48084.80
Name: Close, Length: 150654, dtype: float64


In [50]:
#=============== Parte Manual ===============#
neuronios = 32
act_h = "tanh"
act_out = "linear"
learnr = 0.001
camadas = 4
epochs_n = 30

# Set up neural network model
model = keras.Sequential([
    layers.Dense(neuronios, activation=act_h, input_shape=[8]),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    #layers.Dense(neuronios, activation=act_h),
    #layers.Dense(neuronios, activation=act_h),
    layers.Dense(1,activation=act_h)
  ])

# Compile model
optimizer = Adam(learnr)

model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])

In [51]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 32)                288       
                                                                 
 dense_16 (Dense)            (None, 32)                1056      
                                                                 
 dense_17 (Dense)            (None, 32)                1056      
                                                                 
 dense_18 (Dense)            (None, 32)                1056      
                                                                 
 dense_19 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,489
Trainable params: 3,489
Non-trainable params: 0
_________________________________________________________________


In [52]:
example_batch = normed_train_dataset[:8]
example_result = model.predict(example_batch)
example_result

1/1 [==============================] - 0s 79ms/step


array([[ 0.02181009],
       [ 0.02499402],
       [ 0.0282113 ],
       [ 0.00690017],
       [ 0.10345761],
       [-0.0118586 ],
       [ 0.09251417],
       [-0.01232302]], dtype=float32)

In [53]:
# Set up callbacks
history = History()

# Train model
model.fit(normed_train_dataset, train_labels, verbose=2, batch_size=5, epochs= epochs_n, callbacks=[history])

Epoch 1/30
30131/30131 - 65s - loss: 2480289792.0000 - mae: 48820.0078 - mse: 2480289792.0000 - 65s/epoch - 2ms/step
Epoch 2/30
30131/30131 - 64s - loss: 2480295680.0000 - mae: 48819.8242 - mse: 2480295680.0000 - 64s/epoch - 2ms/step
Epoch 3/30
30131/30131 - 64s - loss: 2480293376.0000 - mae: 48819.8477 - mse: 2480293376.0000 - 64s/epoch - 2ms/step
Epoch 4/30
30131/30131 - 64s - loss: 2480290560.0000 - mae: 48820.0078 - mse: 2480290560.0000 - 64s/epoch - 2ms/step
Epoch 5/30
30131/30131 - 65s - loss: 2480298752.0000 - mae: 48819.6641 - mse: 2480298752.0000 - 65s/epoch - 2ms/step
Epoch 6/30
30131/30131 - 64s - loss: 2480292864.0000 - mae: 48819.8359 - mse: 2480292864.0000 - 64s/epoch - 2ms/step
Epoch 7/30
30131/30131 - 62s - loss: 2480304128.0000 - mae: 48819.8203 - mse: 2480304128.0000 - 62s/epoch - 2ms/step
Epoch 8/30
30131/30131 - 62s - loss: 2480297472.0000 - mae: 48819.8008 - mse: 2480297472.0000 - 62s/epoch - 2ms/step
Epoch 9/30
30131/30131 - 62s - loss: 2480292608.0000 - mae: 4881

In [54]:
# Evaluate model
results = model.evaluate(normed_test_dataset, test_labels)
print("Resultado",results)

# Print average loss over all epochs
print("Average loss:", np.mean(history.history['mse']))

1177/1177 [==============================] - 3s 2ms/step - loss: 2473807104.0000 - mae: 48752.6680 - mse: 2473807104.0000
Resultado [2473807104.0, 48752.66796875, 2473807104.0]
Average loss: 2480295432.5333333


In [55]:
# predictions
y_pred = model.predict(normed_test_dataset)
print(y_pred)

1177/1177 [==============================] - 2s 1ms/step
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [56]:
filename = 'modelmanual_c'+ str(camadas) +'_n'+ str(neuronios) +'_actn'+str(act_h) +'_acto'+str(act_out) +'_lr' + str(learnr) + '_lossmae' +'_ep' + str(epochs_n)
model.save(filename, save_format='tf')

INFO:tensorflow:Assets written to: modelmanual_c4_n32_actntanh_actolinear_lr0.001_lossmae_ep30/assets
